In [1]:
import tensorflow as tf

# Check if TensorFlow can access the GPU
gpu_available = tf.config.list_physical_devices('GPU')

print(gpu_available)


2024-06-03 17:34:26.212887: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-03 17:34:26.249674: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-03 17:34:26.249700: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-03 17:34:26.251008: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-03 17:34:26.257764: I tensorflow/core/platform/cpu_feature_guar

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import sys
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

import deepxde as dde
from deepxde.backend import tf
def get_data(filename, n_train, n_test, sampling_rate=10):
    data = scipy.io.loadmat(filename)
    x = data["x_coor"][:, ::sampling_rate].astype(np.float32)
    y = data["y_coor"][:, ::sampling_rate].astype(np.float32)
    t = data["t"].astype(np.float32)
    Vx = data["Vx"][:, ::sampling_rate].astype(np.float32)
    Vy = data["Vy"][:, ::sampling_rate].astype(np.float32)
    
    u = np.stack((Vx, Vy), axis=-1)  # Shape: (N, k, n_points, 2)

    # Branch: initial velocity field
    branch = u[:, 0, :, :]  # Shape: (N, n_points, 2)

    # Trunk: spatial coordinates grid
    xy = np.stack((x, y), axis=-1).reshape(-1, 2)  # Shape: (n_points, 2)
    
    # Output: velocity field at the final time step
    u_final = u[:, -1, :, :]  # Shape: (N, n_points, 2)
    u_final = u_final.reshape(u_final.shape[0], -1)  # Shape: (N, n_points * 2)

    # Split into training and testing data
    branch_train = branch[:n_train] 
    xy_train = xy
    u_final_train = u_final[:n_train]

    branch_test = branch[-n_test:]
    u_final_test = u_final[-n_test:]

    return (branch_train, xy_train), u_final_train, (branch_test, xy_train), u_final_test